In [12]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [13]:
import gensim

In [441]:
cd ~/twitter_analytics/

/home/ycao/twitter_analytics


In [14]:
w2v=gensim.models.Word2Vec.load('../models/wiki_model/wiki.en.word2vec.model')

In [4]:
w2v.most_similar(['obama'])

[(u'barack', 0.7714914083480835),
 (u'biden', 0.6189278364181519),
 (u'obamamost', 0.5470020771026611),
 (u'clinton', 0.5110719203948975),
 (u'mccain', 0.5068162083625793),
 (u'bush', 0.5049417018890381),
 (u'barackbarack', 0.4921623766422272),
 (u'reagan', 0.48733529448509216),
 (u'jzanalytics', 0.48517102003097534),
 (u'hollande', 0.4836955964565277)]

In [19]:
w2v.most_similar('mug')

[(u'n\xfcr', 0.7315970659255981),
 (u'brn', 0.6835487484931946),
 (u'zol', 0.6782622337341309),
 (u'mnz', 0.6602417230606079),
 (u'rbr', 0.6475198268890381),
 (u'zan', 0.6402051448822021),
 (u'dij', 0.6395502090454102),
 (u'jer', 0.6316919922828674),
 (u'hoc', 0.6307550668716431),
 (u'osc', 0.6206264495849609)]

In [22]:
w2v.most_similar('phone')

[(u'phones', 0.7695544362068176),
 (u'voicemail', 0.6773207187652588),
 (u'handset', 0.6292532086372375),
 (u'smartphone', 0.6179442405700684),
 (u'handsets', 0.6175676584243774),
 (u'skype', 0.6126262545585632),
 (u'friendcaller', 0.6083592176437378),
 (u'mobile', 0.6049913167953491),
 (u'telephone', 0.5931021571159363),
 (u'wallet', 0.5846163034439087)]

In [20]:
class NNExplorer(object):
    def __init__(self, L, eta=.9, r = 1000):
        self.S = set(L) #{k:None for k in L}
        self.L = list(self.S)
        self.candid_set = {k:None for k in L}#set(self.S)
	self.eta = eta
	self.r = r
        self.update_centrality()

    def estimate_centrality(self, wd):
        N = len(self.L)
        if N > self.r:
            rand_anchors = [self.L[idx] for idx in np.random.choice(N, self.r)]
        else:
            rand_anchors = self.L
            
    	return np.std([w2v.similarity(wd, ra) for ra in rand_anchors])
	
    def update_centrality(self):    	
        for k in self.candid_set:
            prev_v = self.candid_set[k]
            new_v = self.estimate_centrality(k)
            if prev_v is None:
                self.candid_set[k] = new_v
            else:
                self.candid_set[k] = self.eta*prev_v + (1-self.eta)*new_v                

    def nn_query(self, word, topn=10):
        wl = w2v.most_similar(word, topn=topn)
        for idx, w in enumerate(wl):
            print idx, w[0], w[1]

        return [w[0] for w in wl]

    def response_add_S(self, wl, yeslist=None, nolist=None):
        if yeslist:
            wl = [wl[idx] for idx in yeslist]

        if nolist:
            wl = [w for idx,w in enumerate(wl) if idx not in nolist]

        wl = list(set([w for w in wl if w not in self.S]))
        print wl
        self.L.extend(wl)            
        self.S.update(wl)        
        self.candid_set.update({w:None for w in wl})
        self.update_centrality()
        
    def nn_prop_explore(self):
        keys = self.candid_set.keys()
        w = keys[np.argmin([self.candid_set[k] for k in keys])]
        #w = np.random.choice(list(self.candid_set), 1)[0]
        centrality=self.candid_set.pop(w)
        print w, centrality
        return w
    
    def add_candidate(self, cw):
        if cw not in self.S:
            raise ValueError("word %s not currently in accepted set." % cw)
        elif cw in self.candid_set:
            raise ValueError("word %s already in candidate set." % cw)
        else:
            self.candid_set[cw] = self.estimate_centrality(cw)            

    def drop(self, ws):
        ws = [w for w in ws if w in self.S]
        for w in ws:
            self.S.remove(w)
            self.L.remove(w)
            if w in self.candid_set:
                self.candid_set.pop(w)
            

In [21]:
#ini_set = ['disease', 'flu', 'sick', 'fever']
ini_set = ['samsung','smartphone','technology', 'electronics', 'android', 'processor', 'sensor', 'display', 'battery']
#ini_set = filter(None, map(str.strip, open('hc_word_picked.txt').readlines()))
explorer=NNExplorer(ini_set)

In [22]:
explorer.L

['smartphone',
 'samsung',
 'battery',
 'sensor',
 'display',
 'electronics',
 'technology',
 'processor',
 'android']

In [23]:
w=explorer.nn_prop_explore()
print w
wl=explorer.nn_query(w)

sensor 0.214116073287
sensor
0 sensors 0.805194556713
1 cmos 0.706283688545
2 accelerometer 0.7048843503
3 autofocus 0.688985466957
4 optical 0.680924475193
5 readout 0.660250842571
6 megapixel 0.659176588058
7 liveview 0.658817410469
8 depthsense 0.654824852943
9 expeed 0.650109171867


In [24]:
yeslist = []
nolist = []
explorer.response_add_S(wl, yeslist=yeslist, nolist=nolist)

[u'expeed', u'accelerometer', u'depthsense', u'readout', u'liveview', u'optical', u'sensors', u'megapixel', u'autofocus', u'cmos']


In [49]:
#explorer.response_add_S(['ebola','swine','avian'], yeslist=[], nolist=[])
explorer.response_add_S(['usb', 'features', 'onyx', 'gold'], yeslist=[], nolist=[])

['usb']


KeyError: 's7'

In [367]:
#explorer.drop(['discussed', 'explored', 'questioned', 'analyzed'])
explorer.drop(['t'])

In [50]:
explorer.candid_set

{'LG': None,
 u'accelerometer': 0.20982611165205445,
 'android': 0.24991434835030948,
 'apple': 0.22017218677068257,
 u'autofocus': 0.23321038408519482,
 'battery': 0.21439119740867535,
 'blackberry': 0.24824712213999864,
 'card': None,
 u'cmos': 0.21931709209669434,
 u'depthsense': 0.20234859960437301,
 'design': None,
 'display': 0.20627900574554167,
 'edge': None,
 'electronics': 0.22555752400463217,
 u'expeed': 0.22451158917261158,
 'expensive': None,
 'features': None,
 'gadget': 0.1927482213219332,
 'galaxy': 0.19770842116131376,
 'gold': None,
 'hate': None,
 'htc': 0.21116841551897428,
 'iphone': 0.23269180124070191,
 'korean': 0.20164651258556385,
 u'liveview': 0.20116170791648202,
 'love': None,
 'market': None,
 u'megapixel': 0.22093286249172092,
 'new': None,
 'news': None,
 'nokia': 0.20031282597979,
 'old': None,
 'one': 0.18511744943632469,
 'onyx': None,
 u'optical': 0.22648301206002031,
 'phone': None,
 'price': None,
 'processor': 0.20841747424337842,
 u'readout': 0.2

In [386]:
#explorer.drop([u'uncovered', u'recuperated', u'discovered', u'extracted', u'excavated', u'unearthed', u'removed', u'salvaged'])

In [51]:
len(explorer.L)

51

In [40]:
explorer.L

['smartphone',
 'samsung',
 'battery',
 'sensor',
 'display',
 'electronics',
 'technology',
 'processor',
 'android',
 u'expeed',
 u'accelerometer',
 u'depthsense',
 u'readout',
 u'liveview',
 u'optical',
 u'sensors',
 u'megapixel',
 u'autofocus',
 u'cmos',
 'release',
 'gadget',
 'galaxy',
 'korean',
 'blackberry',
 'nokia',
 'apple',
 'iphone',
 'htc',
 'one',
 'south',
 'news',
 'price',
 's7',
 'user',
 'phone']

In [52]:
with open('samsung_%d.txt' % len(explorer.L), 'w') as f:
    f.writelines(map(lambda x: x.encode('utf-8'), map(lambda x: x+'\n', explorer.L))                )